# 빠른 시작(QUICKSTART)
##### 기계 학습의 일반적인 작업들을 위한 API 소개
##### 더 자세하게 알아볼려면 각 장의 링크를 참조하면 된다.

## 데이터 작업하기
##### 파이토치에는 데이터 작업을 위한 기본 요소 두가지인 torch.utils.data.DataLoader와 torch.utils.data.Dataset가 있다.
##### Dataset은 샘플과 정답(label)을 저장하고, DataLoader는 Dataset을 순회 가능하 객체(iterable)로 감싼다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

##### PyTorch는 TorchText, TorchVision 및 TorchVision 와 같이 도메인 특화 라이브러리를 데이터셋과 함께 제공하고 있다.
##### torchvision.datasets 모듈은 CIFAR, COCO 등과 같은 다양한 실제 비전 데이터에 대한 Dataset을 포함한다.
##### 해당 튜토리얼에서는 FasionMNIST 데이터셋을 사용한다.
##### 모든 TorchVision Dataset은 샘플과 정답을 각각 변경하기 위한 transform 과 target_transform의 두 인자를 포함한다.

In [2]:
# 공개 데이터셋에서 학습/테스트 데이터를 내려받는다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:00<00:00, 117634305.40it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 4712584.51it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 68989831.01it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6409105.67it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



##### Dataset을 DataLoader의 인자로 전달한다. 이는 데이터셋을 순회 가능한 객체로 감싸고, 자동화된 배치, 샘플링, 섞기 등 다중 프로세스로 데이터 불러오기를 지원한다.
##### 여기서 배치 크기를 64로 정의하는데, 즉 데이터로더 객체의 각 요소는 64개의 특징과 정답을 묶음으로 반환한다는 것이다.

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 모델 만들기
##### PyTorch에서 신경망 모델은 nn.Module을 상속받는 클래스를 생성하여 정의한다.
##### __init__ 함수에서 신경망의 계층들을 정의하고, forward 함수에서 신경망에 데이터를 어떻게 전달하지 지정한다.
##### 가능한 경우 GPU 또는 MPS로 신경망을 이동시켜 연산을 가속한다.

In [22]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# 모델 정의
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델 매개변수 최적화하기
##### 모델을 학습할려면 손실 함수(loss function)와 옵티마이저(optimizer)가 필요하다.

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

##### 각 학습 단계에서 모델은 학습 데이터셋에 대한 예측을 수행하고, 예측 오류를 역전파하여 모델의 매개변수를 조정한다.

In [24]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # 예측 오류 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

##### 모델의 학습이 올바르게 되고 있는지 확인하기 위하여 테스트 데이터셋으로 모델의 성능을 확인한다.

In [25]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [26]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309640  [   64/60000]
loss: 2.294716  [ 6464/60000]
loss: 2.274846  [12864/60000]
loss: 2.263777  [19264/60000]
loss: 2.249261  [25664/60000]
loss: 2.232734  [32064/60000]
loss: 2.227972  [38464/60000]
loss: 2.208043  [44864/60000]
loss: 2.210736  [51264/60000]
loss: 2.164792  [57664/60000]
Test Error: 
 Accuracy: 43.9%, Avg loss: 2.159793 

Epoch 2
-------------------------------
loss: 2.177887  [   64/60000]
loss: 2.161308  [ 6464/60000]
loss: 2.104413  [12864/60000]
loss: 2.113900  [19264/60000]
loss: 2.058275  [25664/60000]
loss: 2.017709  [32064/60000]
loss: 2.037570  [38464/60000]
loss: 1.971423  [44864/60000]
loss: 1.990934  [51264/60000]
loss: 1.894487  [57664/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 1.895664 

Epoch 3
-------------------------------
loss: 1.938250  [   64/60000]
loss: 1.896318  [ 6464/60000]
loss: 1.786916  [12864/60000]
loss: 1.821373  [19264/60000]
loss: 1.694942  [25664/60000]
loss: 1.673306  [32064/600

## 모델 저장하기
##### 모델을 저장하는 일반적인 방법은 내부 상태 사전을 직렬화하는 것이다. (모델의 매개변수들을 포함)

In [27]:
torch.save(model.state_dict(), "model.pth")
print("saved")

saved


## 모델 불러오기
##### 모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함된다.

In [28]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [29]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
